# Tutorial : Train and deploy a rasa chatbot

The goal of this tutorial is to train and deploy a simple rasa chatbot. To do this, once you have clone the repo git, you can directly go in the folder rasa_bot and then we will train the model to have a model saved in the folder models. 

## Install the requirements

The project was created in a venv environment. To add it in a jupyter environment, you must install pip and the file requirements_rasa.txt. Run this for simplification. 

In [4]:
%conda install pip
%pip install --no-cache-dir -r requirements_rasa.txt

ValueError: The python kernel does not appear to be a conda environment.  Please use ``%pip install`` instead.